In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import udf
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from IPython.display import display
from ipywidgets import interact
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from operator import add
import sys
import numpy as np
import pandas as pd
import time
import datetime
from pyspark.mllib.tree import RandomForest, RandomForestModel

from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext("local",'app')
spark = SparkSession.builder.appName('name').config('spark.sql.shuffle.partitions',10).getOrCreate()

# 1. load data
## 1.1 big trainset

In [2]:
data=spark.read.csv('train_test/train_flight.csv',header=True,inferSchema=True)
data.printSchema()

root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- NEW_SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- NEW_DAY: integer (nullable = true)



## 1.2 small_trainset

In [55]:
airlines=data.select('AIRLINE').distinct().rdd.map(lambda row:row['AIRLINE']).collect()
airlines

[u'MQ',
 u'NK',
 u'OO',
 u'B6',
 u'F9',
 u'US',
 u'WN',
 u'VX',
 u'AA',
 u'DL',
 u'UA',
 u'EV',
 u'AS',
 u'HA']

In [6]:
data_df=data.toPandas()
data_df.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,NEW_SCHEDULED_DEPARTURE,NEW_DAY
0,1,AA,ABQ,CLT,28,-12,205,187,1449,605,535,-30,40,201
1,1,AA,ABQ,CLT,31,-9,205,185,1449,605,536,-29,40,215
2,1,AA,ABQ,CLT,31,-9,205,194,1449,605,545,-20,40,243
3,1,AA,ABQ,CLT,34,-6,205,197,1449,605,551,-14,40,208
4,1,AA,ABQ,CLT,36,-4,205,213,1449,605,609,4,40,229


In [25]:
data_df.describe()

,DAY_OF_WEEK,DEPARTURE_TIME,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,NEW_SCHEDULED_DEPARTURE,NEW_DAY,DEPARTURE_DELAY,square_schedule,square_day,schedule_by_day
count,479982.000000,479982.000000,479982.000000,479982.000000,479982.000000,479982.000000,479982.000000,479982.000000,479982.000000,479982.000000,479982.000000,4.799820e+05,479982.000000,479982.000000
mean,3.947198,1299.554883,172.461207,167.099289,1047.994879,1502.547106,1489.613533,-3.427633,791.441200,198.402188,1.934285,7.146360e+05,49968.049387,156880.277765
std,1.990313,499.976158,77.905314,77.405218,636.651491,510.987688,518.135968,19.024725,297.080736,102.978849,12.899076,4.849673e+05,40496.916305,105326.948777
min,1.000000,1.000000,42.000000,34.000000,83.000000,1.000000,1.000000,-87.000000,5.000000,1.000000,-68.000000,2.500000e+01,1.000000,10.000000
25%,2.000000,854.000000,114.000000,110.000000,599.000000,1112.000000,1104.000000,-15.000000,535.000000,115.000000,-5.000000,2.862250e+05,13225.000000,74730.000000
50%,4.000000,1301.000000,163.000000,157.000000,980.000000,1525.000000,1518.000000,-7.000000,780.000000,207.000000,-3.000000,6.084000e+05,42849.000000,138990.000000
75%,6.000000,1719.000000,210.000000,205.000000,1345.000000,1925.000000,1922.000000,5.000000,1035.000000,270.000000,3.000000,1.071225e+06,72900.000000,224790.000000
max,7.000000,2400.000000,523.000000,630.000000,3784.000000,2359.000000,2400.000000,225.000000,1439.000000,365.000000,59.000000,2.070721e+06,133225.000000,525235.000000


In [82]:
%matplotlib inline
data_small=data_df[['DEPARTURE_DELAY','NEW_DAY',"NEW_SCHEDULED_DEPARTURE"]]
data_join=data_small.join(pd.get_dummies(data_df['ORIGIN_AIRPORT'],prefix='AIRPORT'))
feature_col=list(data_join.columns)
feature_col.remove('DEPARTURE_DELAY')
labels=data_join['DEPARTURE_DELAY'].as_matrix()
X=data_join[feature_col].as_matrix()
print labels.shape,X.shape

(505811,) (505811, 100)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
scaler = StandardScaler()
X_train=scaler.fit_transform(X)

poly = PolynomialFeatures(degree = 2)
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
X_ = poly.fit_transform(X_train)
regr.fit(X_, labels)
result = regr.predict(X_)
score =mean_squared_error(result, labels)
print("Mean squared error = ", score)

In [ ]:
print X_.shape

## 1.3 choose one dataset

In [3]:
#dataset=data_small
dataset_unprocess1=data
print(dataset_unprocess1.count())
dataset_unprocess1.printSchema()
#dataset.select("SCHEDULED_TIME",'schedule_departure').show()

4185551
root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- NEW_SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- NEW_DAY: integer (nullable = true)



In [4]:
# just try airport clustering
dataset_unprocess2=dataset_unprocess1.filter(dataset_unprocess1['AIRLINE']=='AS')
dataset_unprocess2=dataset_unprocess2.filter(dataset_unprocess2['DEPARTURE_DELAY']<60)
dataset_unprocess2.count()

122036

# 0.1 change label to classification

# 0.2 change label to doubletype


In [5]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import DoubleType,IntegerType

dataset=dataset_unprocess2

udf = UserDefinedFunction(lambda x: x*1.0, DoubleType())
new_data=dataset.select('*',udf(dataset['DEPARTURE_DELAY']).alias('double_labels'))
dataset=new_data.drop('DEPARTURE_DELAY')
dataset=dataset.withColumnRenamed('double_labels','DEPARTURE_DELAY')
dataset.printSchema()
# create a new feature square_schedule
udf = UserDefinedFunction(lambda x: x*x, IntegerType())
new_data=dataset.select('*',udf(dataset['NEW_SCHEDULED_DEPARTURE']).alias('square_schedule'))
dataset=new_data

#create a new feature square_day
udf = UserDefinedFunction(lambda x: x*x, IntegerType())
new_data=dataset.select('*',udf(dataset['NEW_DAY']).alias('square_day'))
dataset=new_data

# create a new feature schedule_by_day
new_data=dataset.select('*',(dataset['NEW_DAY']*dataset['NEW_SCHEDULED_DEPARTURE']).alias('schedule_by_day'))
dataset=new_data
dataset.printSchema()

root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- NEW_SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- NEW_DAY: integer (nullable = true)
 |-- DEPARTURE_DELAY: double (nullable = true)

root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nu

# 2 feature transformation pipeline
## 2.1 feature selection (can be updated)

In [25]:
# dataset=data
categoricalColumns = []  # to add
numericCols = ['NEW_SCHEDULED_DEPARTURE',"square_day","square_schedule","NEW_DAY","DISTANCE","DAY_OF_WEEK"]  # to add
# all_features=categoricalColumns+numericCols

## 2.2 transform and onehot

In [26]:
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
cols=dataset.columns

stages = [] 
feature_names=[]
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, 
        outputCol=categoricalCol+"Index")
    encoder = OneHotEncoder(inputCol=categoricalCol+"Index", 
        outputCol=categoricalCol+"classVec")
    stages += [stringIndexer, encoder]

assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="Features")
stages += [assembler]
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(dataset)
dataset_transformed = pipelineModel.transform(dataset)
pipelineModel.write().overwrite().save("test_pipeline")
from pyspark.ml.feature import PolynomialExpansion
from pyspark.mllib.linalg import Vectors

px = PolynomialExpansion(degree=1, inputCol="Features", outputCol="features")
polyDF = px.transform(dataset_transformed)

selectedcols = ['DEPARTURE_DELAY', "features"] 
dataset_transformed = polyDF.select(selectedcols)
dataset_transformed=dataset_transformed.select('*').withColumnRenamed('DEPARTURE_DELAY','label')
dataset_transformed.printSchema()
dataset_transformed.select('*').show(truncate=False)

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

+-----+-------------------------------------------+
|label|features                                   |
+-----+-------------------------------------------+
|-18.0|[980.0,49284.0,960400.0,222.0,1180.0,1.0]  |
|-14.0|[980.0,52441.0,960400.0,229.0,1180.0,1.0]  |
|-12.0|[980.0,37636.0,960400.0,194.0,1180.0,1.0]  |
|-12.0|[980.0,40401.0,960400.0,201.0,1180.0,1.0]  |
|-11.0|[980.0,43264.0,960400.0,208.0,1180.0,1.0]  |
|-8.0 |[980.0,32400.0,960400.0,180.0,1180.0,1.0]  |
|-8.0 |[980.0,46225.0,960400.0,215.0,1180.0,1.0]  |
|-5.0 |[980.0,34969.0,960400.0,187.0,1180.0,1.0]  |
|-8.0 |[1000.0,59049.0,1000000.0,243.0,1180.0,1.0]|
|-6.0 |[1000.0,55696.0,1000000.0,236.0,1180.0,1.0]|
|-12.0|[495.0,32400.0,245025.0,180.0,253.0,1.0]   |
|-9.0 |[495.0,55696.0,245025.0,236.0,253.0,1.0]   |
|-8.0 |[495.0,40401.0,245025.0,201.0,253.0,1.0]   |
|-2.0 |[495.0,46225.0,245025.0,215.0,253.0,1.0]   |
|0.0  |[495.0,59049.0,245025.0,24

In [9]:
testDataset=dataset_transformed
testDataset.count()

30388

## 2.3 sample and split dataset into trainingData and testData

In [17]:
#dataset_used,left_behind=dataset_transformed.randomSplit((0.3,0.7),1)
trainingData,testData=dataset_transformed.randomSplit((0.7,0.3),1)
print trainingData.count()
print testData.count()

85622
36414


## 2.7 feature importances

In [27]:
from pyspark.ml.regression import RandomForestRegressor
rf= RandomForestRegressor(numTrees=10, maxDepth=3, seed=42)
model = rf.fit(dataset_transformed)
print(assemblerInputs)
model.featureImportances

['NEW_SCHEDULED_DEPARTURE', 'square_day', 'square_schedule', 'NEW_DAY', 'DISTANCE', 'DAY_OF_WEEK']


SparseVector(6, {0: 0.4367, 1: 0.0583, 2: 0.2718, 3: 0.1039, 4: 0.1193, 5: 0.0101})

In [58]:
for i in range(len(assemblerInputs)):
    print "Feature Name:"+str(assemblerInputs[i])+"\t importances:"+str(model.featureImportances[i])

Feature Name:NEW_SCHEDULED_DEPARTURE	 importances:0.436664050418
Feature Name:square_day	 importances:0.0583377762333
Feature Name:square_schedule	 importances:0.271783558573
Feature Name:NEW_DAY	 importances:0.103858173085
Feature Name:DISTANCE	 importances:0.11925619668
Feature Name:DAY_OF_WEEK	 importances:0.0101002450105


# 4 machine learning model(by pyspark.ml.)

# 4.1 train model (RandomForest)

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
numFolds =3

rf = RandomForestRegressor(labelCol="label", featuresCol="features")   
paramGrid = ParamGridBuilder()\
    .addGrid(rf.numTrees,[50]) \
    .build()
crossval = CrossValidator(
    estimator=rf,
    estimatorParamMaps=paramGrid,
    evaluator=RegressionEvaluator(),
    numFolds=numFolds)
model = crossval.fit(trainingData)

## 4.2 linear family model

In [27]:
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import GeneralizedLinearRegression
# Isotonic
model = IsotonicRegression(labelCol="label", featuresCol="features").fit(trainingData)

#linear regression
#lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=1,labelCol="label", featuresCol="features")
#model = lr.fit(trainingData)

# G LR, gaussian,
#glr = GeneralizedLinearRegression(family="Tweedie", maxIter=10, regParam=0.3,labelCol="label", featuresCol="features")
#model = glr.fit(trainingData)
#model.write().overwrite().save('name'+'_model')

## 4.2 performance

In [28]:
from pyspark.mllib.evaluation import RegressionMetrics
cvModel=model
trainPredictionsAndLabels = cvModel.transform(trainingData).select("label", "prediction").rdd
validPredictionsAndLabels = cvModel.transform(testData).select("label", "prediction").rdd
trainRegressionMetrics = RegressionMetrics(trainPredictionsAndLabels)
validRegressionMetrics = RegressionMetrics(validPredictionsAndLabels)

bestModel = cvModel
#featureImportances = bestModel.featureImportances.toArray()
#print (featureImportances)

output = str("\n=====================================================================\n" +
      "TrainingData count: {0}\n".format(trainingData.count()) +
      "TestData count: {0}\n".format(testData.count()) +
      "=====================================================================\n" +
      "Training data MSE = {}\n".format(trainRegressionMetrics.meanSquaredError) +
      "Training data RMSE = {}\n".format(trainRegressionMetrics.rootMeanSquaredError) +
      "Training data R-squared = {}\n".format(trainRegressionMetrics.r2) +
      "Training data MAE = {}\n".format(trainRegressionMetrics.meanAbsoluteError) +
      #"Training data Explained variance = {}\n".format(trainRegressionMetrics.explainedVariance) +
      "=====================================================================\n" +
      "Validation data MSE = {0}\n".format(validRegressionMetrics.meanSquaredError) +
      "Validation data RMSE = {0}\n".format(validRegressionMetrics.rootMeanSquaredError) +
      "Validation data R-squared = {0}\n".format(validRegressionMetrics.r2) +
      "Validation data MAE = {0}\n".format(validRegressionMetrics.meanAbsoluteError) +
      #"Validation data Explained variance = {0}\n".format(validRegressionMetrics.explainedVariance) +
     # "=====================================================================\n" +
      #"CV params explained: {}\n".format(cvModel.explainParams()) +
     # "RandomForest params explained: {}\n".format(bestModel.explainParams()) +
      #"RandomForest features importances:\n {0}\n".format("\n".join(map(lambda z: "{0} = {1}".format(str(z[0]),str(z[1])), zip(featureCols, featureImportances)))) +
"=====================================================================\n")
print(output)


TrainingData count: 85622
TestData count: 36414
Training data MSE = 365.319181986
Training data RMSE = 19.1133247235
Training data R-squared = -1.49562158888
Training data MAE = 15.4465324333
Validation data MSE = 372.668039765
Validation data RMSE = 19.3046118781
Validation data R-squared = -1.53560656357
Validation data MAE = 15.5763717252



In [29]:
#trainPredictionsAndLabels = cvModel.transform(trainingData).select("label", "prediction").rdd
testPredictionsAndLabels = cvModel.transform(testDataset).select("label", "prediction").rdd

testRegressionMetrics = RegressionMetrics(testPredictionsAndLabels)
output = str("\n=====================================================================\n" +
      "testing data MSE = {0}\n".format(testRegressionMetrics.meanSquaredError) +
      "testing data RMSE = {0}\n".format(testRegressionMetrics.rootMeanSquaredError) +
      "testing data R-squared = {0}\n".format(testRegressionMetrics.r2) +
      "testing data MAE = {0}\n".format(testRegressionMetrics.meanAbsoluteError) +
"=====================================================================\n")
print(output)


testing data MSE = 364.903810715
testing data RMSE = 19.10245562
testing data R-squared = -1.48519198925
testing data MAE = 15.4240818744



# 3. ML model (by MLlib)
## 3.1 generate RDD

In [28]:
# change into RDD
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml import linalg as ml_linalg

def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))
        
airlineRDD=dataset_transformed.rdd.map(lambda row: LabeledPoint(row['label'],as_mllib(row['features'])))

## 3.2 split trainset and testset 

In [29]:
#  Spliting dataset into train and test dtasets
airlineRDD.cache()
use_data,left_data=airlineRDD.randomSplit([0.2,0.8])
trainingData,testData=use_data.randomSplit([0.8,0.2])

## 3.3 use Random Forest classifier

In [30]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                        numTrees=100, featureSubsetStrategy="auto",
                                        impurity='variance', maxDepth=10, maxBins=32)
# Evaluate model on test instances and compute test error
#predictions = model.predict(testData.map(lambda x: x.features))
#labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
#testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
#       float(testData.count())
#print('Test Mean Squared Error = ' + str(testMSE))

In [44]:
from pyspark.mllib.evaluation import RegressionMetrics
print('training performance===========')
predictions = model.predict(trainingData.map(lambda x: x.features)).map(lambda element:float(element))
labelsAndPredictions = trainingData.map(lambda lp: lp.label).zip(predictions)
# Instantiate metrics object
metrics = RegressionMetrics(labelsAndPredictions)
# Squared Error
print("MSE = %s" % metrics.meanSquaredError)
print("RMSE = %s" % metrics.rootMeanSquaredError)
 # R-squared
print("R-squared = %s" % metrics.r2)
# Mean absolute error
print("MAE = %s" % metrics.meanAbsoluteError)

print("testing===============")
predictions = model.predict(testData.map(lambda x: x.features)).map(lambda element:float(element))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
# Instantiate metrics object
metrics = RegressionMetrics(labelsAndPredictions)
# Squared Error
print("MSE = %s" % metrics.meanSquaredError)
print("RMSE = %s" % metrics.rootMeanSquaredError)
 # R-squared
print("R-squared = %s" % metrics.r2)
# Mean absolute error
print("MAE = %s" % metrics.meanAbsoluteError)

training performance===========
MSE = 154.176016496
RMSE = 12.4167635274
R-squared = -13.1173920111
MAE = 8.37304414901
testing===============
MSE = 153.607064852
RMSE = 12.393831726
R-squared = -12.9510277354
MAE = 8.43209812783


## 3.4 use GBDT

In [41]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
model = GradientBoostedTrees.trainRegressor(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=200)

## 3.6 Ridge Regression

In [34]:
from pyspark.mllib.regression import LabeledPoint,RidgeRegressionWithSGD
# Build the model
model = RidgeRegressionWithSGD.train(trainingData, iterations=100, step=0.000001)

In [19]:
# Evaluate the model on training data
valuesAndPreds = trainingData.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = nan


## 3.3 model training

In [45]:
# train models
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
model = LinearRegressionWithSGD.train(trainRDD, iterations=100, step=0.0000001)

/home/andy/spark-2.2.0-bin-hadoop2.7/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


## 3.4 model evaluation

In [ ]:
# Evaluate the model on training data
valuesAndPreds = testRDD.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

##  3.5 save model

In [ ]:
# Save and load model
model.save(sc, "model/pythonLinearRegressionWithSGDModel")

sameModel = LinearRegressionModel.load(sc, "model/pythonLinearRegressionWithSGDModel")